# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
from sqlalchemy import create_engine
import nltk

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df.message
y = df[df.columns[4:]]
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('moc', MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('moc',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)

In [9]:
y_pred = pipeline.predict(X_test)

for i in range(36):
    print(y_test.columns[i])
    # y_test is a dataframe so i can use iloc but y_pred is an array and array does not have iloc()
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print('---------------------------------------------')      

related
              precision    recall  f1-score   support

           0       0.76      0.28      0.41      1530
           1       0.81      0.97      0.88      4973
           2       0.46      0.59      0.52        51

    accuracy                           0.80      6554
   macro avg       0.68      0.61      0.60      6554
weighted avg       0.80      0.80      0.77      6554

---------------------------------------------
request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5423
           1       0.84      0.46      0.59      1131

    accuracy                           0.89      6554
   macro avg       0.87      0.72      0.76      6554
weighted avg       0.89      0.89      0.88      6554

---------------------------------------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6537
           1       0.00      0.00      0.00        17

    accuracy  

/Users/fuchunyang/Library/Python/3.11/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fuchunyang/Library/Python/3.11/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fuchunyang/Library/Python/3.11/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
    'moc__estimator__n_estimators': [10, 50],
    'moc__estimator__min_samples_split': [2, 5]
    #'vect__max_features': [2,3,4,10,20],
    #'tfidf__norm': ['l1', 'l2'],  # Example: Additional parameter for TfidfTransformer
    #'moc__estimator__max_depth': [None, 5, 10] 
}


In [11]:
gscv = GridSearchCV(pipeline, param_grid=parameters)
gscv.fit(X_train, y_train)

In [ ]:
print(gscv.best_estimator_)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('moc',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=50)))])


In [ ]:
print(gscv.best_estimator_)
best_model = gscv.best_estimator_

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('moc',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=50)))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = best_model.predict(X_test)

for i in range(36):
    print(y_test.columns[i])
    # y_test is a dataframe so i can use iloc but y_pred is an array and array does not have iloc()
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print('---------------------------------------------')  

related
              precision    recall  f1-score   support

           0       0.71      0.29      0.41      1535
           1       0.81      0.96      0.88      4980
           2       0.35      0.46      0.40        39

    accuracy                           0.80      6554
   macro avg       0.62      0.57      0.56      6554
weighted avg       0.79      0.80      0.77      6554

---------------------------------------------
request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5418
           1       0.88      0.42      0.57      1136

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.75      6554
weighted avg       0.89      0.89      0.87      6554

---------------------------------------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy  

/Users/fuchunyang/Library/Python/3.11/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fuchunyang/Library/Python/3.11/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fuchunyang/Library/Python/3.11/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(best_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.